In [1]:
!pip install Flask pyngrok

In [3]:
"""
CSCI210 Final Project - Rock-Paper-Scissors Tournament
Google Colab Version

Author: Nathan Anderson
Date: December 18, 2024

This application implements a Flask-based web application for managing a
multi-player Rock-Paper-Scissors tournament. It demonstrates the use of
Python data structures (Dictionary, List) and algorithms (Sorting, Searching).

Data Structures:
    - LEADERBOARD: Dictionary for O(1) player lookups and updates
    - game_state: Dictionary for current game state management
    - Sorted lists: For dual-view leaderboard display

API Endpoints:
    - POST /api/player/register: Register new player
    - POST /api/game/start: Initialize 10-round game
    - POST /api/game/play_round: Execute one round
    - GET /api/leaderboard: Retrieve sorted leaderboard
    - GET /api/game/state: Get current game state
"""

# Standard library imports
import random
import threading
import time

# Third-party imports
from flask import Flask, request, jsonify
from pyngrok import ngrok


# Initialize Flask application
app = Flask(__name__)


# ============================================================================
# GLOBAL DATA STRUCTURES
# ============================================================================

# Central data store - Dictionary for O(1) player lookups
# Structure: {player_name: {"score": int, "games_won": int}}
LEADERBOARD = {}

# Current game state - tracks active game information
game_state = {
    "player1": None,           # Player 1 name (string)
    "player2": None,           # Player 2 name (string)
    "player1_score": 0,        # Player 1 rounds won in current game
    "player2_score": 0,        # Player 2 rounds won in current game
    "rounds_played": 0,        # Number of rounds completed (max 10)
    "game_active": False,      # Whether a game is currently in progress
    "last_winner": None        # Winner of last game (for retention)
}


# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def determine_winner(choice1, choice2):
    """
    Determine the winner of a Rock-Paper-Scissors round.

    Uses standard RPS rules:
        - Rock beats Scissors
        - Scissors beats Paper
        - Paper beats Rock

    Args:
        choice1 (str): Player 1's choice ("rock", "paper", or "scissors")
        choice2 (str): Player 2's choice ("rock", "paper", or "scissors")

    Returns:
        str: "player1" if Player 1 wins,
             "player2" if Player 2 wins,
             "tie" if choices are identical

    Time Complexity: O(1)
    """
    # Handle tie case
    if choice1 == choice2:
        return "tie"

    # Define winning combinations as a set for O(1) lookup
    winning_combinations = {
        ("rock", "scissors"),
        ("scissors", "paper"),
        ("paper", "rock")
    }

    # Check if Player 1's choice beats Player 2's choice
    if (choice1, choice2) in winning_combinations:
        return "player1"
    else:
        return "player2"


def get_sorted_leaderboard():
    """
    Convert LEADERBOARD dictionary to two sorted list views.

    Implements the required sorting algorithms:
        1. Alphabetical sorting by player name
        2. Descending sorting by cumulative score

    Returns:
        dict: Dictionary containing two sorted lists:
            {
                "by_name": [...],   # Sorted alphabetically
                "by_score": [...]   # Sorted by score (descending)
            }

    Time Complexity: O(n log n) due to sorting operations
    Space Complexity: O(n) for creating list copies
    """
    # Convert dictionary to list of dictionaries for sorting
    leaderboard_list = []
    for player_name, stats in LEADERBOARD.items():
        leaderboard_list.append({
            "name": player_name,
            "score": stats["score"],
            "games_won": stats["games_won"]
        })

    # Sort by name (alphabetically ascending)
    sorted_by_name = sorted(leaderboard_list, key=lambda x: x["name"])

    # Sort by score (numerically descending)
    sorted_by_score = sorted(
        leaderboard_list,
        key=lambda x: x["score"],
        reverse=True
    )

    return {
        "by_name": sorted_by_name,
        "by_score": sorted_by_score
    }


# ============================================================================
# HTML TEMPLATE
# ============================================================================

HTML_TEMPLATE = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Rock-Paper-Scissors Tournament</title>
    <style>
        /* Reset default styles */
        * { margin: 0; padding: 0; box-sizing: border-box; }

        /* Main body styling */
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
        }

        /* Container for centering content */
        .container { max-width: 1200px; margin: 0 auto; }

        /* Main heading */
        h1 {
            text-align: center;
            color: white;
            margin-bottom: 30px;
            font-size: 2.5em;
            text-shadow: 2px 2px 4px rgba(0,0,0,0.3);
        }

        /* Section containers */
        .game-section, .leaderboard-section {
            background: white;
            border-radius: 15px;
            padding: 30px;
            margin-bottom: 30px;
            box-shadow: 0 10px 30px rgba(0,0,0,0.3);
        }

        /* Player input grid */
        .player-setup {
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 20px;
            margin-bottom: 20px;
        }

        /* Player input styling */
        .player-input { display: flex; flex-direction: column; }
        .player-input label {
            font-weight: bold;
            margin-bottom: 8px;
            color: #333;
        }
        .player-input input {
            padding: 12px;
            font-size: 16px;
            border: 2px solid #ddd;
            border-radius: 8px;
            transition: border-color 0.3s;
        }
        .player-input input:focus {
            outline: none;
            border-color: #667eea;
        }
        .player-input input:disabled {
            background-color: #f0f0f0;
            cursor: not-allowed;
        }

        /* Button group layout */
        .button-group {
            display: flex;
            gap: 15px;
            justify-content: center;
            margin: 20px 0;
        }

        /* Button base styles */
        button {
            padding: 12px 30px;
            font-size: 16px;
            font-weight: bold;
            border: none;
            border-radius: 8px;
            cursor: pointer;
            transition: all 0.3s;
            box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        }
        button:hover {
            transform: translateY(-2px);
            box-shadow: 0 6px 12px rgba(0,0,0,0.2);
        }
        button:disabled {
            opacity: 0.5;
            cursor: not-allowed;
            transform: none;
        }

        /* Button color variants */
        .btn-start { background: #10b981; color: white; }
        .btn-rock, .btn-paper, .btn-scissors {
            background: #667eea;
            color: white;
            flex: 1;
        }

        /* Game status display */
        .game-status {
            text-align: center;
            padding: 20px;
            background: #f8f9fa;
            border-radius: 8px;
            margin: 20px 0;
        }

        /* Score display grid */
        .score-display {
            display: flex;
            justify-content: space-around;
            margin: 20px 0;
            font-size: 1.5em;
        }
        .score-item { text-align: center; }
        .score-value {
            font-size: 2em;
            font-weight: bold;
            color: #667eea;
        }

        /* Round result message */
        .round-result {
            text-align: center;
            padding: 15px;
            background: #e0e7ff;
            border-radius: 8px;
            margin: 15px 0;
            font-size: 1.1em;
        }

        /* Leaderboard grid layout */
        .leaderboard-container {
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 30px;
        }
        .leaderboard-view {
            background: #f8f9fa;
            padding: 20px;
            border-radius: 10px;
        }
        .leaderboard-view h3 {
            color: #667eea;
            margin-bottom: 15px;
            text-align: center;
        }

        /* Leaderboard table styling */
        .leaderboard-table {
            width: 100%;
            border-collapse: collapse;
        }
        .leaderboard-table th {
            background: #667eea;
            color: white;
            padding: 12px;
            text-align: left;
        }
        .leaderboard-table td {
            padding: 10px 12px;
            border-bottom: 1px solid #ddd;
        }
        .leaderboard-table tr:hover { background: #e0e7ff; }

        /* Message box styling */
        .message {
            padding: 12px;
            margin: 15px 0;
            border-radius: 8px;
            text-align: center;
        }
        .message.success {
            background: #d1fae5;
            color: #065f46;
        }
        .message.error {
            background: #fee2e2;
            color: #991b1b;
        }

        /* Utility class */
        .hidden { display: none; }
    </style>
</head>
<body>
    <div class="container">
        <h1>🎮 Rock-Paper-Scissors Tournament 🎮</h1>

        <!-- Game Section -->
        <div class="game-section">
            <h2>Game Setup</h2>

            <!-- Player Name Inputs -->
            <div class="player-setup">
                <div class="player-input">
                    <label for="player1">Player 1:</label>
                    <input type="text" id="player1"
                           placeholder="Enter Player 1 name">
                </div>
                <div class="player-input">
                    <label for="player2">Player 2:</label>
                    <input type="text" id="player2"
                           placeholder="Enter Player 2 name">
                </div>
            </div>

            <!-- Start Game Button -->
            <div class="button-group">
                <button class="btn-start" onclick="startGame()">
                    Start New Game
                </button>
            </div>

            <!-- Message Display Area -->
            <div id="message" class="message hidden"></div>

            <!-- Game Play Area (Hidden Until Game Starts) -->
            <div id="gamePlay" class="hidden">
                <!-- Round Counter -->
                <div class="game-status">
                    <h3>Round <span id="roundNumber">0</span> of 10</h3>
                </div>

                <!-- Current Score Display -->
                <div class="score-display">
                    <div class="score-item">
                        <div id="player1Name">Player 1</div>
                        <div class="score-value" id="player1Score">0</div>
                    </div>
                    <div class="score-item">
                        <div id="player2Name">Player 2</div>
                        <div class="score-value" id="player2Score">0</div>
                    </div>
                </div>

                <!-- Choice Selection Panels -->
                <div style="display: grid; grid-template-columns: 1fr 1fr;
                            gap: 30px; margin: 20px 0;">
                    <!-- Player 1 Choice Panel -->
                    <div style="text-align: center; padding: 20px;
                                background: #f8f9fa; border-radius: 10px;">
                        <h3 style="margin-bottom: 15px;"
                            id="player1NameLabel">Player 1</h3>
                        <div class="button-group"
                             style="flex-direction: column;">
                            <button class="btn-rock"
                                    onclick="selectChoice(1, 'rock')"
                                    id="p1-rock">✊ Rock</button>
                            <button class="btn-paper"
                                    onclick="selectChoice(1, 'paper')"
                                    id="p1-paper">✋ Paper</button>
                            <button class="btn-scissors"
                                    onclick="selectChoice(1, 'scissors')"
                                    id="p1-scissors">✌️ Scissors</button>
                        </div>
                        <div id="player1Choice" style="margin-top: 15px;
                             font-size: 1.2em; font-weight: bold;
                             color: #667eea;">
                            Waiting...
                        </div>
                    </div>

                    <!-- Player 2 Choice Panel -->
                    <div style="text-align: center; padding: 20px;
                                background: #f8f9fa; border-radius: 10px;">
                        <h3 style="margin-bottom: 15px;"
                            id="player2NameLabel">Player 2</h3>
                        <div class="button-group"
                             style="flex-direction: column;">
                            <button class="btn-rock"
                                    onclick="selectChoice(2, 'rock')"
                                    id="p2-rock">✊ Rock</button>
                            <button class="btn-paper"
                                    onclick="selectChoice(2, 'paper')"
                                    id="p2-paper">✋ Paper</button>
                            <button class="btn-scissors"
                                    onclick="selectChoice(2, 'scissors')"
                                    id="p2-scissors">✌️ Scissors</button>
                        </div>
                        <div id="player2Choice" style="margin-top: 15px;
                             font-size: 1.2em; font-weight: bold;
                             color: #667eea;">
                            Waiting...
                        </div>
                    </div>
                </div>

                <!-- Play Round Button -->
                <div class="button-group">
                    <button class="btn-start" onclick="playRound()"
                            id="playRoundBtn" disabled
                            style="opacity: 0.5; font-size: 1.2em;
                                   padding: 15px 40px;">
                        Play Round
                    </button>
                </div>

                <!-- Round Result Display -->
                <div id="roundResult" class="round-result hidden"></div>
            </div>
        </div>

        <!-- Leaderboard Section -->
        <div class="leaderboard-section">
            <h2 style="text-align: center; margin-bottom: 20px;">
                Tournament Leaderboard
            </h2>
            <div class="leaderboard-container">
                <!-- Leaderboard Sorted by Name -->
                <div class="leaderboard-view">
                    <h3>Sorted by Name</h3>
                    <table class="leaderboard-table">
                        <thead>
                            <tr>
                                <th>Player</th>
                                <th>Score</th>
                                <th>Games Won</th>
                            </tr>
                        </thead>
                        <tbody id="leaderboardByName">
                            <tr>
                                <td colspan="3" style="text-align: center;">
                                    No data yet
                                </td>
                            </tr>
                        </tbody>
                    </table>
                </div>

                <!-- Leaderboard Sorted by Score -->
                <div class="leaderboard-view">
                    <h3>Sorted by Score</h3>
                    <table class="leaderboard-table">
                        <thead>
                            <tr>
                                <th>Player</th>
                                <th>Score</th>
                                <th>Games Won</th>
                            </tr>
                        </thead>
                        <tbody id="leaderboardByScore">
                            <tr>
                                <td colspan="3" style="text-align: center;">
                                    No data yet
                                </td>
                            </tr>
                        </tbody>
                    </table>
                </div>
            </div>
        </div>
    </div>

    <script>
        // ====================================================================
        // GLOBAL STATE VARIABLES
        // ====================================================================

        let currentGameState = null;     // Stores current game information
        let player1Selection = null;     // Player 1's choice this round
        let player2Selection = null;     // Player 2's choice this round

        // ====================================================================
        // INITIALIZATION
        // ====================================================================

        /**
         * Initialize the application on page load
         * - Load existing leaderboard data
         * - Check for winner retention from previous game
         */
        window.onload = function() {
            loadLeaderboard();
            checkGameState();
        };

        // ====================================================================
        // GAME STATE MANAGEMENT
        // ====================================================================

        /**
         * Check if there's a previous game winner to retain as Player 1
         * Implements the winner retention requirement
         */
        async function checkGameState() {
            try {
                const response = await fetch('/api/game/state');
                const data = await response.json();

                // If last game had a winner, auto-fill Player 1
                if (data.last_winner) {
                    document.getElementById('player1').value =
                        data.last_winner;
                    document.getElementById('player1').disabled = true;
                }
            } catch (error) {
                console.error('Error checking game state:', error);
            }
        }

        /**
         * Start a new 10-round game between two players
         * Validates input and calls the API to initialize game state
         */
        async function startGame() {
            // Get and validate player names
            const player1 = document.getElementById('player1').value.trim();
            const player2 = document.getElementById('player2').value.trim();

            if (!player1 || !player2) {
                showMessage('Please enter both player names', 'error');
                return;
            }

            if (player1 === player2) {
                showMessage('Players must have different names', 'error');
                return;
            }

            try {
                // Call API to start game
                const response = await fetch('/api/game/start', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({
                        player1: player1,
                        player2: player2
                    })
                });

                const data = await response.json();

                if (data.success) {
                    // Initialize UI for new game
                    showMessage(data.message, 'success');
                    document.getElementById('gamePlay')
                        .classList.remove('hidden');
                    document.getElementById('player1Name').textContent =
                        player1;
                    document.getElementById('player2Name').textContent =
                        player2;
                    document.getElementById('player1NameLabel').textContent =
                        player1;
                    document.getElementById('player2NameLabel').textContent =
                        player2;
                    document.getElementById('player1Score').textContent = '0';
                    document.getElementById('player2Score').textContent = '0';
                    document.getElementById('roundNumber').textContent = '1';
                    document.getElementById('roundResult')
                        .classList.add('hidden');
                    currentGameState = data.game_state;
                    resetChoices();
                } else {
                    showMessage(data.message, 'error');
                }
            } catch (error) {
                showMessage('Error starting game: ' + error.message, 'error');
            }
        }

        // ====================================================================
        // PLAYER CHOICE HANDLING
        // ====================================================================

        /**
         * Handle player choice selection
         * Updates UI and enables "Play Round" button when both have chosen
         *
         * @param {number} player - Player number (1 or 2)
         * @param {string} choice - Choice made ("rock", "paper", "scissors")
         */
        function selectChoice(player, choice) {
            if (player === 1) {
                // Store Player 1's selection
                player1Selection = choice;
                document.getElementById('player1Choice').textContent =
                    '✓ Choice locked in';
                document.getElementById('player1Choice').style.color =
                    '#10b981';

                // Highlight selected button
                ['rock', 'paper', 'scissors'].forEach(c => {
                    const btn = document.getElementById(`p1-${c}`);
                    if (c === choice) {
                        btn.style.background = '#10b981';
                        btn.style.transform = 'scale(1.05)';
                    } else {
                        btn.style.background = '#667eea';
                        btn.style.transform = 'scale(1)';
                    }
                });
            } else {
                // Store Player 2's selection
                player2Selection = choice;
                document.getElementById('player2Choice').textContent =
                    '✓ Choice locked in';
                document.getElementById('player2Choice').style.color =
                    '#10b981';

                // Highlight selected button
                ['rock', 'paper', 'scissors'].forEach(c => {
                    const btn = document.getElementById(`p2-${c}`);
                    if (c === choice) {
                        btn.style.background = '#10b981';
                        btn.style.transform = 'scale(1.05)';
                    } else {
                        btn.style.background = '#667eea';
                        btn.style.transform = 'scale(1)';
                    }
                });
            }

            // Enable "Play Round" button if both players have chosen
            if (player1Selection && player2Selection) {
                const playBtn = document.getElementById('playRoundBtn');
                playBtn.disabled = false;
                playBtn.style.opacity = '1';
            }
        }

        /**
         * Reset choices for the next round
         * Clears selections and resets UI elements
         */
        function resetChoices() {
            player1Selection = null;
            player2Selection = null;

            // Reset choice status text
            document.getElementById('player1Choice').textContent = 'Waiting...';
            document.getElementById('player1Choice').style.color = '#667eea';
            document.getElementById('player2Choice').textContent = 'Waiting...';
            document.getElementById('player2Choice').style.color = '#667eea';

            // Reset all buttons to default state
            ['rock', 'paper', 'scissors'].forEach(c => {
                document.getElementById(`p1-${c}`).style.background =
                    '#667eea';
                document.getElementById(`p1-${c}`).style.transform =
                    'scale(1)';
                document.getElementById(`p2-${c}`).style.background =
                    '#667eea';
                document.getElementById(`p2-${c}`).style.transform =
                    'scale(1)';
            });

            // Disable "Play Round" button
            const playBtn = document.getElementById('playRoundBtn');
            playBtn.disabled = true;
            playBtn.style.opacity = '0.5';
        }

        // ====================================================================
        // ROUND EXECUTION
        // ====================================================================

        /**
         * Execute one round of Rock-Paper-Scissors
         * Sends both players' choices to API and handles response
         */
        async function playRound() {
            // Validate both players have made choices
            if (!player1Selection || !player2Selection) {
                showMessage('Both players must make a choice!', 'error');
                return;
            }

            try {
                // Send choices to API
                const response = await fetch('/api/game/play_round', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({
                        player1_choice: player1Selection,
                        player2_choice: player2Selection
                    })
                });

                const data = await response.json();

                if (data.success) {
                    // Handle game completion (10 rounds done)
                    if (data.game_complete) {
                        handleGameCompletion(data);
                    } else {
                        // Handle mid-game round result
                        handleRoundResult(data);
                    }
                } else {
                    showMessage(data.message, 'error');
                }
            } catch (error) {
                showMessage('Error playing round: ' + error.message, 'error');
            }
        }

        /**
         * Handle the completion of a 10-round game
         * Updates final scores, determines winner, updates leaderboard
         *
         * @param {Object} data - API response data
         */
        function handleGameCompletion(data) {
            // Update final scores
            document.getElementById('player1Score').textContent =
                data.final_score.player1;
            document.getElementById('player2Score').textContent =
                data.final_score.player2;

            // Display game winner
            let resultText = 'Game Over! ';
            if (data.game_winner === 'tie') {
                resultText += "It's a tie!";
            } else {
                resultText += `${data.game_winner} wins the game!`;
            }

            document.getElementById('roundResult').textContent = resultText;
            document.getElementById('roundResult').classList.remove('hidden');

            // Hide game controls
            document.getElementById('gamePlay').classList.add('hidden');

            // Refresh leaderboard
            loadLeaderboard();

            // Set up next game with winner retention
            if (data.last_winner) {
                document.getElementById('player1').value = data.last_winner;
                document.getElementById('player1').disabled = true;
                document.getElementById('player2').value = '';
                document.getElementById('player2').disabled = false;
            } else {
                // Tie game - both fields editable
                document.getElementById('player1').disabled = false;
                document.getElementById('player2').disabled = false;
            }
        }

        /**
         * Handle result of a single round (not game end)
         * Updates scores and prepares for next round
         *
         * @param {Object} data - API response data
         */
        function handleRoundResult(data) {
            // Update current scores
            document.getElementById('player1Score').textContent =
                data.current_score.player1;
            document.getElementById('player2Score').textContent =
                data.current_score.player2;
            document.getElementById('roundNumber').textContent =
                data.round_result.round_number + 1;

            // Build result message
            let resultText = `Round ${data.round_result.round_number}: `;
            resultText +=
                `${document.getElementById('player1').value} chose ` +
                `${player1Selection}, `;
            resultText +=
                `${document.getElementById('player2').value} chose ` +
                `${player2Selection}. `;

            // Add round winner
            if (data.round_result.round_winner === 'tie') {
                resultText += "It's a tie!";
            } else if (data.round_result.round_winner === 'player1') {
                resultText +=
                    `${document.getElementById('player1').value} ` +
                    'wins this round!';
            } else {
                resultText +=
                    `${document.getElementById('player2').value} ` +
                    'wins this round!';
            }

            // Display result
            document.getElementById('roundResult').textContent = resultText;
            document.getElementById('roundResult').classList.remove('hidden');

            // Auto-reset for next round after 2 seconds
            setTimeout(() => {
                resetChoices();
                document.getElementById('roundResult').classList.add('hidden');
            }, 2000);
        }

        // ====================================================================
        // LEADERBOARD MANAGEMENT
        // ====================================================================

        /**
         * Load and display the leaderboard
         * Fetches sorted data from API and updates both views
         */
        async function loadLeaderboard() {
            try {
                const response = await fetch('/api/leaderboard');
                const data = await response.json();

                if (data.success) {
                    // Update leaderboard sorted by name
                    updateLeaderboardTable(
                        'leaderboardByName',
                        data.leaderboard.by_name
                    );

                    // Update leaderboard sorted by score
                    updateLeaderboardTable(
                        'leaderboardByScore',
                        data.leaderboard.by_score
                    );
                }
            } catch (error) {
                console.error('Error loading leaderboard:', error);
            }
        }

        /**
         * Update a leaderboard table with player data
         *
         * @param {string} tableId - ID of table body element to update
         * @param {Array} players - Array of player objects
         */
        function updateLeaderboardTable(tableId, players) {
            const tableBody = document.getElementById(tableId);
            tableBody.innerHTML = '';

            if (players.length === 0) {
                // Show "no data" message
                tableBody.innerHTML =
                    '<tr><td colspan="3" style="text-align: center;">' +
                    'No data yet</td></tr>';
            } else {
                // Populate table with player data
                players.forEach(player => {
                    const row = `<tr>
                        <td>${player.name}</td>
                        <td>${player.score}</td>
                        <td>${player.games_won}</td>
                    </tr>`;
                    tableBody.innerHTML += row;
                });
            }
        }

        // ====================================================================
        // USER FEEDBACK
        // ====================================================================

        /**
         * Display a temporary message to the user
         *
         * @param {string} message - Message text to display
         * @param {string} type - Message type ('success' or 'error')
         */
        function showMessage(message, type) {
            const messageDiv = document.getElementById('message');
            messageDiv.textContent = message;
            messageDiv.className = `message ${type}`;
            messageDiv.classList.remove('hidden');

            // Auto-hide after 5 seconds
            setTimeout(() => {
                messageDiv.classList.add('hidden');
            }, 5000);
        }
    </script>
</body>
</html>
"""


# ============================================================================
# API ENDPOINTS
# ============================================================================

@app.route('/')
def index():
    """
    Serve the main game page.

    Returns:
        str: HTML template for the game interface
    """
    return HTML_TEMPLATE


@app.route('/api/player/register', methods=['POST'])
def register_player():
    """
    Register a new player in the LEADERBOARD dictionary.

    Creates a new player entry if they don't already exist.
    If player exists, returns their current statistics.

    Expected JSON Request:
        {
            "player_name": "string"
        }

    Returns:
        tuple: (JSON response, HTTP status code)
            Success (201): New player registered
            Success (200): Player already exists
            Error (400): Invalid input
    """
    data = request.get_json()
    player_name = data.get('player_name', '').strip()

    # Validate player name is not empty
    if not player_name:
        return jsonify({
            "success": False,
            "message": "Player name cannot be empty"
        }), 400

    # Check if player already exists
    if player_name in LEADERBOARD:
        return jsonify({
            "success": True,
            "message": f"Player '{player_name}' already registered",
            "player": {
                "name": player_name,
                "score": LEADERBOARD[player_name]["score"],
                "games_won": LEADERBOARD[player_name]["games_won"]
            }
        }), 200

    # Register new player with initial statistics
    LEADERBOARD[player_name] = {"score": 0, "games_won": 0}

    return jsonify({
        "success": True,
        "message": f"Player '{player_name}' registered successfully",
        "player": {"name": player_name, "score": 0, "games_won": 0}
    }), 201


@app.route('/api/game/start', methods=['POST'])
def start_game():
    """
    Initialize a new 10-round game between two players.

    Validates player names and initializes game_state dictionary.
    Automatically registers players in LEADERBOARD if they don't exist.

    Expected JSON Request:
        {
            "player1": "string",
            "player2": "string"
        }

    Returns:
        tuple: (JSON response, HTTP status code)
            Success (200): Game started successfully
            Error (400): Invalid input or validation failure
    """
    data = request.get_json()
    player1_name = data.get('player1', '').strip()
    player2_name = data.get('player2', '').strip()

    # Validate both player names are provided
    if not player1_name or not player2_name:
        return jsonify({
            "success": False,
            "message": "Both player names are required"
        }), 400

    # Validate player names are different
    if player1_name == player2_name:
        return jsonify({
            "success": False,
            "message": "Players must have different names"
        }), 400

    # Register players if they don't exist in LEADERBOARD
    for player in [player1_name, player2_name]:
        if player not in LEADERBOARD:
            LEADERBOARD[player] = {"score": 0, "games_won": 0}

    # Initialize game state
    game_state["player1"] = player1_name
    game_state["player2"] = player2_name
    game_state["player1_score"] = 0
    game_state["player2_score"] = 0
    game_state["rounds_played"] = 0
    game_state["game_active"] = True

    return jsonify({
        "success": True,
        "message": f"Game started between {player1_name} and {player2_name}",
        "game_state": {
            "player1": game_state["player1"],
            "player2": game_state["player2"],
            "player1_score": 0,
            "player2_score": 0,
            "rounds_played": 0,
            "rounds_total": 10
        }
    }), 200


@app.route('/api/game/play_round', methods=['POST'])
def play_round():
    """
    Execute one round of Rock-Paper-Scissors.

    Determines round winner, updates scores, and checks for game completion.
    After 10 rounds, updates LEADERBOARD with final statistics.

    Expected JSON Request:
        {
            "player1_choice": "rock" | "paper" | "scissors",
            "player2_choice": "rock" | "paper" | "scissors"
        }

    Returns:
        tuple: (JSON response, HTTP status code)
            Success (200): Round executed successfully
            Error (400): Invalid input or no active game
    """
    # Validate game is active
    if not game_state["game_active"]:
        return jsonify({
            "success": False,
            "message": "No active game. Please start a new game first."
        }), 400

    # Get and validate player choices
    data = request.get_json()
    player1_choice = data.get('player1_choice', '').lower()
    player2_choice = data.get('player2_choice', '').lower()

    valid_choices = ['rock', 'paper', 'scissors']
    if (player1_choice not in valid_choices or
            player2_choice not in valid_choices):
        return jsonify({
            "success": False,
            "message": "Invalid choice. Must be 'rock', 'paper', or 'scissors'"
        }), 400

    # Determine round winner using helper function
    round_winner = determine_winner(player1_choice, player2_choice)

    # Update round scores
    if round_winner == "player1":
        game_state["player1_score"] += 1
    elif round_winner == "player2":
        game_state["player2_score"] += 1

    game_state["rounds_played"] += 1

    # Check if game is complete (10 rounds played)
    game_complete = game_state["rounds_played"] >= 10

    if game_complete:
        return handle_game_completion(
            player1_choice,
            player2_choice,
            round_winner
        )
    else:
        return handle_round_continuation(
            player1_choice,
            player2_choice,
            round_winner
        )


def handle_game_completion(player1_choice, player2_choice, round_winner):
    """
    Handle game completion after 10 rounds.

    Updates LEADERBOARD with final scores and game results.
    Determines overall game winner and sets up winner retention.

    Args:
        player1_choice (str): Player 1's final round choice
        player2_choice (str): Player 2's final round choice
        round_winner (str): Winner of the final round

    Returns:
        tuple: (JSON response, HTTP status code)
    """
    # Determine overall game winner
    if game_state["player1_score"] > game_state["player2_score"]:
        game_winner = game_state["player1"]
    elif game_state["player2_score"] > game_state["player1_score"]:
        game_winner = game_state["player2"]
    else:
        game_winner = "tie"

    # Get player names
    player1_name = game_state["player1"]
    player2_name = game_state["player2"]

    # Update LEADERBOARD with cumulative scores
    LEADERBOARD[player1_name]["score"] += game_state["player1_score"]
    LEADERBOARD[player2_name]["score"] += game_state["player2_score"]

    # Update games won counter and set last winner for retention
    if game_winner == player1_name:
        LEADERBOARD[player1_name]["games_won"] += 1
        game_state["last_winner"] = player1_name
    elif game_winner == player2_name:
        LEADERBOARD[player2_name]["games_won"] += 1
        game_state["last_winner"] = player2_name
    else:
        # Tie game - no winner retention
        game_state["last_winner"] = None

    # Mark game as inactive
    game_state["game_active"] = False

    return jsonify({
        "success": True,
        "round_result": {
            "player1_choice": player1_choice,
            "player2_choice": player2_choice,
            "round_winner": round_winner,
            "round_number": game_state["rounds_played"]
        },
        "game_complete": True,
        "game_winner": game_winner,
        "final_score": {
            "player1": game_state["player1_score"],
            "player2": game_state["player2_score"]
        },
        "last_winner": game_state["last_winner"]
    }), 200


def handle_round_continuation(player1_choice, player2_choice, round_winner):
    """
    Handle continuation of game after a non-final round.

    Returns round result and current game state.

    Args:
        player1_choice (str): Player 1's choice this round
        player2_choice (str): Player 2's choice this round
        round_winner (str): Winner of this round

    Returns:
        tuple: (JSON response, HTTP status code)
    """
    return jsonify({
        "success": True,
        "round_result": {
            "player1_choice": player1_choice,
            "player2_choice": player2_choice,
            "round_winner": round_winner,
            "round_number": game_state["rounds_played"]
        },
        "game_complete": False,
        "current_score": {
            "player1": game_state["player1_score"],
            "player2": game_state["player2_score"]
        },
        "rounds_remaining": 10 - game_state["rounds_played"]
    }), 200


@app.route('/api/leaderboard', methods=['GET'])
def get_leaderboard():
    """
    Retrieve the complete leaderboard with sorted views.

    Converts LEADERBOARD dictionary to sorted lists using sorting algorithms.
    Provides two views: alphabetically by name and numerically by score.

    Returns:
        tuple: (JSON response, HTTP status code)
            Success (200): Returns sorted leaderboard data
    """
    sorted_data = get_sorted_leaderboard()

    return jsonify({
        "success": True,
        "leaderboard": sorted_data,
        "total_players": len(LEADERBOARD)
    }), 200


@app.route('/api/game/state', methods=['GET'])
def get_game_state():
    """
    Return current game state information.

    Useful for checking if a game is active and retrieving
    the last winner for winner retention functionality.

    Returns:
        tuple: (JSON response, HTTP status code)
            Success (200): Returns current game state
    """
    return jsonify({
        "success": True,
        "game_active": game_state["game_active"],
        "player1": game_state["player1"],
        "player2": game_state["player2"],
        "player1_score": game_state["player1_score"],
        "player2_score": game_state["player2_score"],
        "rounds_played": game_state["rounds_played"],
        "last_winner": game_state["last_winner"]
    }), 200


# ============================================================================
# GOOGLE COLAB SPECIFIC SETUP
# ============================================================================

def run_flask():
    """
    Run the Flask application.
    """
    app.run(port=5000, use_reloader=False)


# Start Flask server in background thread
print("Starting Flask server on port 5000...")
flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

# Wait for Flask to initialize
time.sleep(3)

# Set ngrok authentication token
ngrok.set_auth_token("371ncNdEtc4Pmuz9ScUTLxHqKAo_3N5dd9RTkjgRXgyNQF93x")

# Create ngrok tunnel to expose local server
print("Creating public URL with ngrok...")
public_url = ngrok.connect(5000)

# Display access information
print("\n" + "=" * 60)
print(f"🎮 Your app is running at: {public_url}")
print("=" * 60)
print("\nClick the link above to open your Rock-Paper-Scissors Tournament!")
print("Note: Keep this Colab cell running while you play the game.")
print("\n" + "=" * 60)

Starting Flask server on port 5000...
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


Creating public URL with ngrok...

🎮 Your app is running at: NgrokTunnel: "https://brittani-hydropic-preeffectually.ngrok-free.dev" -> "http://localhost:5000"

Click the link above to open your Rock-Paper-Scissors Tournament!
Note: Keep this Colab cell running while you play the game.

